In [3]:
import sys
sys.path.insert(1, ".")

from Utils import Utils

In [4]:
vocab = Utils.file_to_vocab("./data/en-train.txt")




In [5]:
# how many unique words are there in train?
#19850
# setwords = list(set(words))
# for i in range(10):
#     print(setwords[i])



In [11]:
from scipy import spatial

with open("./data/en-val.txt",'r') as f1:
    val = f1.readlines()
    
results = []
for idx, line in enumerate(val):
    segments = line.split('\t')
    first_seg = segments[0].split()
    second_seg = segments[1].split()
    
    first_seg_vec = Utils.bow_to_vec(vocab, set(first_seg))
    second_seg_vec = Utils.bow_to_vec(vocab, set(second_seg))
    result = (1 - spatial.distance.cosine(first_seg_vec, second_seg_vec)) * 5
    results.append(result)
    
    if idx % 100 == 0:
        print(idx, result)
    
    

#[print(results[i]) for i in range(10)]


0 3.5355339059327373
100 3.9528470752104736
200 2.357022603955159
300 2.551551815399144
400 4.225771273642583
500 2.8603877677367775
600 3.608439182435161
700 2.7277236279499046
800 1.1180339887498947
900 2.721655269759087
1000 1.666666666666667
1100 4.008918628686366
1200 4.008918628686366
1300 3.333333333333334
1400 0.0


In [12]:
with open('simple_baseline.txt', 'w') as f:      
    for score in results:
        f.write("{}\n".format(score)) 

In [10]:
spatial.distance.cosine([1,1,1], [1,1,1])

0.0